
# Examples of using Spark 

In [ ]:
from azureml.core import Workspace, Experiment, Environment,ScriptRunConfig
# from azureml.widgets import RunDetails
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.environment import Environment
import time

In [ ]:
from ray_on_aml.core import Ray_On_AML

ws = Workspace.from_config()
ray_on_aml =Ray_On_AML(ws=ws, compute_cluster ="d12-v2-ssh", maxnode=2)
ray = ray_on_aml.getRay()
ray.cluster_resources()

# Reading Delta Lake data from ADLS Gen2 

In [ ]:
storage_account_name ="adlsdatalakegen6"
storage_account_key=ws.get_default_keyvault().get_secret("adlsdatalakegen6")
additional_spark_configs ={f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net":f"{storage_account_key}"}
spark = ray_on_aml.getSpark(executor_cores =3,num_executors =3 ,executor_memory='10GB', additional_spark_configs=additional_spark_configs)
#Number of nodes (including head node) can be set as number of executor. 

In [ ]:
adls_data = spark.read.format("delta").load("abfss://mltraining@adlsdatalakegen6.dfs.core.windows.net/ISDWeatherDelta")
adls_data.groupby("stationName").count().head(20)


## Synapse SQL Pool Data Access

In [ ]:
server_name = "jdbc:sqlserver://sy2qwhqqkv7eacsws1.sql.azuresynapse.net:1433"
database_name = "sy2qwhqqkv7eacsws1p1"
url = server_name + ";" + "databaseName=" + database_name + ";"

table_name = "ISDWeatherDelta"
username = ""
password = "" # Please specify password here or get it from keyvault


jdbcDF = spark.read \
        .format("jdbc") \
        .option("url", url) \
        .option("dbtable", table_name) \
        .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
        .option("user", username) \
        .option("password", password).load()

jdbcDF.show()